# Load Libraries and packages

In [11]:
import pandas as pd
import csv
import os
import sys

#### Weather data

In [2]:
# -----------------------------------------------------------------
# 1) Read the county-to-climdivs mapping file into a lookup dict
#    The file has 3 columns (POSTAL_FIPS_ID, NCDC_FIPS_ID, CLIMDIV_ID)
#    We'll map NCDC_FIPS_ID -> (POSTAL_FIPS_ID, CLIMDIV_ID)
# -----------------------------------------------------------------

mapping = {}
with open(os.path.join("county-to-climdivs.txt"), "r") as f:
    next(f)  # skip header line if it exists
    for line in f:
        parts = line.strip().split()
        if len(parts) != 3:
            continue
        postal_fips, ncdc_fips, climdiv_id = parts
        mapping[ncdc_fips] = (postal_fips, climdiv_id)

# -----------------------------------------------------------------
#2) Define a helper function to parse each line in tmaxcy/pcpncy
# -----------------------------------------------------------------
def parse_clim_line(line):
    """
    Given a line (string) from tmaxcy or pcpncy,
    returns a dict with raw_code, state, county, division, year, and the 12 monthly values.
    If the line can't be mapped (NCDC FIPS not found), return None.
    """
    parts = line.strip().split()
    if len(parts) < 13:
        return None  # not enough data
    
    # The first item is the 11-digit code: e.g. "01001271895"
    code = parts[0]
    monthly_values = parts[1:]  # the next 12 numbers
    
    ncdc_fips = code[:5]       #first 5 digits
    data_type = code[5:7]      #next 2 digits (27 for tmax, 01 for pcpn)
    year = code[7:]            #last 4 digits
    
    if ncdc_fips not in mapping:
        return None
    
    postal_fips, climdiv_id = mapping[ncdc_fips]
    # postal_fips e.g. "04001" => correct_state="04", correct_county="001"
    correct_state = postal_fips[:2]
    correct_county = postal_fips[2:]
    # climdiv_id e.g. "0202" => last two digits "02" for division
    division = climdiv_id[-2:]
    
    # Create a dict with the data, including the raw_code for clarity
    return {
        "raw_code": code,
        "state": correct_state,
        "county": correct_county,
        "division": division,
        "year": year,
        "Jan": monthly_values[0],
        "Feb": monthly_values[1],
        "Mar": monthly_values[2],
        "Apr": monthly_values[3],
        "May": monthly_values[4],
        "Jun": monthly_values[5],
        "Jul": monthly_values[6],
        "Aug": monthly_values[7],
        "Sep": monthly_values[8],
        "Oct": monthly_values[9],
        "Nov": monthly_values[10],
        "Dec": monthly_values[11]
    }

# -----------------------------------------------------------------
#3) Read & parse tmaxcy (temperature) lines
# -----------------------------------------------------------------
tmax_records = []
with open(os.path.join("240924 climdiv-tmaxcy-v1.0.0-20240906.txt"), "r") as f:
    for line in f:
        parsed = parse_clim_line(line)
        if parsed:
            tmax_records.append(parsed)

df_tmax = pd.DataFrame(tmax_records)
print("TMAX DataFrame:\n", df_tmax.head())

# -----------------------------------------------------------------
#4) Lastly Read & parse pcpncy (precipitation) lines
# -----------------------------------------------------------------
pcpn_records = []
with open(os.path.join("240924 climdiv-pcpncy-v1.0.0-20240906.txt"), "r") as f:
    for line in f:
        parsed = parse_clim_line(line)
        if parsed:
            pcpn_records.append(parsed)

df_pcpn = pd.DataFrame(pcpn_records)
print("PCPN DataFrame:\n", df_pcpn.head())


TMAX DataFrame:
       raw_code state county division  year    Jan    Feb    Mar    Apr    May  \
0  01001271895    01    001       03  1895  53.70  48.70  67.60  76.40  81.90   
1  01001271896    01    001       03  1896  54.20  60.80  65.30  81.60  88.50   
2  01001271897    01    001       03  1897  54.20  63.10  71.40  75.10  83.20   
3  01001271898    01    001       03  1898  60.60  59.10  71.00  72.00  89.50   
4  01001271899    01    001       03  1899  55.60  53.40  68.80  73.40  89.30   

     Jun    Jul    Aug    Sep    Oct    Nov    Dec  
0  89.20  91.10  90.40  90.90  76.00  66.60  58.00  
1  88.20  92.00  94.50  90.80  77.20  69.90  58.70  
2  95.60  93.30  89.90  88.90  81.30  68.10  58.80  
3  93.90  91.50  88.80  86.70  73.60  61.70  55.70  
4  93.70  92.20  92.60  87.50  78.40  68.10  56.60  
PCPN DataFrame:
       raw_code state county division  year   Jan   Feb    Mar   Apr   May  \
0  01001011895    01    001       03  1895  7.03  2.96   8.36  3.53  3.96   
1  0100

#### Check for missing data

In [3]:
# Check for missing data in temperature and precipitation DataFrames
#identify monthly columns by suffix in temperature and precipitation DataFrames
tmax_months = [col for col in df_tmax.columns if col.endswith("_tmax")]
pcpn_months = [col for col in df_pcpn.columns if col.endswith("_pcpn")]

#count missing values in temperature data (-99.99 indicates missing)
missing_tmax = (df_tmax[tmax_months] == -99.99).sum()
print("Missing values in Temperature Data (per month):")
print(missing_tmax)

#count missing values in precipitation data (-9.99 indicates missing)
missing_pcpn = (df_pcpn[pcpn_months] == -9.99).sum()
print("\nMissing values in Precipitation Data (per month):")
print(missing_pcpn)

Missing values in Temperature Data (per month):
Series([], dtype: float64)

Missing values in Precipitation Data (per month):
Series([], dtype: float64)


# Align with corn yield data

In [7]:
# Load corn yield data


# Join path and filename correctly
path_corn = "../_corn_yield_data/"
filename = "df_corn_yield.csv"
full_path = os.path.join(path_corn, filename)


df_corn_yield = pd.read_csv(
    full_path,
    dtype={
        'state_ansi': str,
        'county_ansi': str,
        'district_code': str
    }
)

print(df_corn_yield.head())

   year state_ansi county_ansi district_code  \
0  2023         17         107            04   
1  2023         17         115            04   
2  2023         17         125            04   
3  2023         17         113            04   
4  2023         17         129            04   

                                    data_item  value   cv  
0  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE  211.1  1.6  
1  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE  225.3  2.8  
2  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE  208.1  3.6  
3  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE  223.3  2.2  
4  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE  214.4  3.9  


### Start pre-merging diagnostics

In [8]:
# Filter the temperature and precipitation DataFrames to include only states in the corn yield data
# Here we reuse the unique_states from the crop yield DataFrame as the allowed state list
allowed_states = df_corn_yield["state_ansi"].astype(str).unique()

df_tmax = df_tmax[df_tmax["state"].isin(allowed_states)]
df_pcpn = df_pcpn[df_pcpn["state"].isin(allowed_states)]

print("Filtered Temperature DataFrame States:", df_tmax["state"].unique())
print("Filtered Precipitation DataFrame States:", df_pcpn["state"].unique())

Filtered Temperature DataFrame States: ['17' '19' '27' '31']
Filtered Precipitation DataFrame States: ['17' '19' '27' '31']


In [9]:
# Count unique counties per state in the temperature DataFrame:
tmax_counties_per_state = df_tmax.groupby('state')['county'].nunique()
print("Unique counties per state in df_tmax:")
print(tmax_counties_per_state)

# Count unique counties per state in the corn yield DataFrame:
corn_counties_per_state = df_corn_yield.groupby('state_ansi')['county_ansi'].nunique()
print("\nUnique counties per state in df_corn_yield:")
print(corn_counties_per_state)

Unique counties per state in df_tmax:
state
17    102
19     99
27     87
31     93
Name: county, dtype: int64

Unique counties per state in df_corn_yield:
state_ansi
17    102
19     99
27     85
31     93
Name: county_ansi, dtype: int64


In [10]:
# ------------------------------------------------------------------
## --- CHECK HIGHEST MIN AND LOWEST MAX YEAR FOR CORN YIELD DATA ---
# Compute the min and max year for each state in the corn yield dataset
state_year_ranges = df_corn_yield.groupby("state_ansi")["year"].agg(["min", "max"])
print("Year ranges per state:")
print(state_year_ranges)

# Get the highest minimum year (i.e. the maximum of the minimum years)
highest_min_year = state_year_ranges["min"].max()

# Get the lowest maximum year (i.e. the minimum of the maximum years)
lowest_max_year = state_year_ranges["max"].min()

print(f"Highest minimum year across states: {highest_min_year}")
print(f"Lowest maximum year across states: {lowest_max_year}")


# ------------------------------------------------------------------
## --- APPLY TO WEATHER DATA ---
# Convert the 'year' column to int (if not already) and filter the data frames
df_tmax["year"] = df_tmax["year"].astype(int)
df_tmax = df_tmax[(df_tmax["year"] >= highest_min_year) & (df_tmax["year"] <= lowest_max_year)]

df_pcpn["year"] = df_pcpn["year"].astype(int)
df_pcpn = df_pcpn[(df_pcpn["year"] >= highest_min_year) & (df_pcpn["year"] <= lowest_max_year)]

print("Filtered df_tmax:")
print(df_tmax.head(100000))
print("\nFiltered df_pcpn:")
print(df_pcpn.head(100000))

Year ranges per state:
             min   max
state_ansi            
17          1926  2023
19          1926  2023
27          1926  2023
31          1926  2023
Highest minimum year across states: 1926
Lowest maximum year across states: 2023
Filtered df_tmax:
           raw_code state county division  year    Jan    Feb    Mar    Apr  \
72831   11001271926    17    001       03  1926  37.00  43.10  45.00  57.00   
72832   11001271927    17    001       03  1927  32.80  46.10  53.90  62.00   
72833   11001271928    17    001       03  1928  36.60  42.00  53.80  60.20   
72834   11001271929    17    001       03  1929  29.70  31.80  56.80  64.40   
72835   11001271930    17    001       03  1930  26.20  49.20  51.20  70.00   
...             ...   ...    ...      ...   ...    ...    ...    ...    ...   
222554  25185272019    31    185       06  2019  34.00  27.30  42.40  65.40   
222555  25185272020    31    185       06  2020  33.70  41.90  53.00  63.10   
222556  25185272021    31    

### Make growing season variables

In [12]:
# Be aware:
# - Get params from config file
# - convert to SI units

import calendar
# Add parent directory to Python path
sys.path.append(os.path.abspath(".."))
# Now import config
import config

# This creates the list of standard month abbreviations
# e.g., ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_name_list = calendar.month_abbr 

# This uses the start and end month numbers from your config to slice the list
# GS_START_MONTH (e.g., 4) becomes the start of the slice
# GS_END_MONTH + 1 (e.g., 9 + 1 = 10) becomes the end of the slice (exclusive)
months = list(month_name_list[config.GS_START_MONTH : config.GS_END_MONTH + 1])
print(months)

# --- Temperature Calculation ---
# Convert the monthly temperature columns to numeric values
df_tmax[months] = df_tmax[months].apply(pd.to_numeric, errors='coerce')
# Convert Fahrenheit to Celsius: °C = (°F - 32) × 5/9
df_tmax[months] = (df_tmax[months] - 32) * 5 / 9
# Compute the arithmetic mean (across the months) for each row
df_tmax['temp_gs'] = df_tmax[months].mean(axis=1)
# Group by year, state, county, and division to get one value per group
df_temp_growing_season = df_tmax.groupby(
    ['year', 'state', 'county', 'division']
)['temp_gs'].mean().reset_index()

print("Growing season temperature by year, state, county, and division:")
print(df_temp_growing_season.head())

# --- Precipitation Calculation ---
# Convert the monthly precipitation columns to numeric values
df_pcpn[months] = df_pcpn[months].apply(pd.to_numeric, errors='coerce')
# Convert inches to millimeters: mm = inches × 25.4
df_pcpn[months] = df_pcpn[months] * 25.4
# Compute the total precipitation (sum over the months) for each row
df_pcpn['pcpn_gs'] = df_pcpn[months].sum(axis=1)
# Group by year, state, county, and division to get one value per group
df_precip_growing_season = df_pcpn.groupby(
    ['year', 'state', 'county', 'division']
)['pcpn_gs'].mean().reset_index()

print("\nGrowing season precipitation by year, state, county, and division:")
print(df_precip_growing_season.head())

['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
Growing season temperature by year, state, county, and division:
   year state county division    temp_gs
0  1926    17    001       03  25.472222
1  1926    17    003       08  27.944444
2  1926    17    005       06  26.324074
3  1926    17    007       02  22.277778
4  1926    17    009       03  25.222222

Growing season precipitation by year, state, county, and division:
   year state county division  pcpn_gs
0  1926    17    001       03  831.342
1  1926    17    003       08  497.840
2  1926    17    005       06  610.616
3  1926    17    007       02  708.660
4  1926    17    009       03  861.314


#### Check one more time if nothing is missing

In [13]:
# Use the corn yield year range as the common complete years
complete_years = set(range(highest_min_year, lowest_max_year + 1))

# Get the unique (state, county) keys present in either temperature or precipitation datasets
keys_temp = set(df_temp_growing_season.groupby(["state", "county"]).groups.keys())
keys_pcpn = set(df_precip_growing_season.groupby(["state", "county"]).groups.keys())
all_keys = keys_temp.union(keys_pcpn)

missing_weather_report = {}

for key in all_keys:
    state, county = key
    # Get years available from the temperature dataset for the county (if any)
    years_temp = set(
        df_temp_growing_season.loc[
            (df_temp_growing_season["state"] == state) & (df_temp_growing_season["county"] == county),
            "year"
        ].astype(int).unique()
    )
    # Get years available from the precipitation dataset for the county (if any)
    years_pcpn = set(
        df_precip_growing_season.loc[
            (df_precip_growing_season["state"] == state) & (df_precip_growing_season["county"] == county),
            "year"
        ].astype(int).unique()
    )
    
    # Combine the available years from both data types
    years_present = years_temp.union(years_pcpn)
    missing_rows = sorted(complete_years - years_present)
    
    if missing_rows:
        missing_weather_report[(state, county)] = missing_rows

# Print the missing report in sorted order if any missing data exists, else print 'no missing data'
if missing_weather_report:
    for state, county in sorted(missing_weather_report.keys()):
        print(f"State {state}, County {county}:")
        print(f"  Rows missing for years: {missing_weather_report[(state, county)]}")
else:
    print("no missing data")

no missing data


### Merge data

In [14]:
# Example: Load your dataframes (replace these with your actual file loading commands)
# corn_yield_df = pd.read_csv("corn_yield.csv")
# df_temp_growing_season = pd.read_csv("temp_growing_season.csv")
# df_precip_growing_season = pd.read_csv("precip_growing_season.csv")

# Rename columns in corn_yield_df to match the keys in the other dataframes
corn_yield_df = df_corn_yield.rename(columns={
    'state_ansi': 'state',
    'county_ansi': 'county',
    'district_code': 'division'
}).copy()

# Ensure that the key columns 'state', 'county', and 'year' have the same data types
# In this example, we'll convert 'state' and 'county' to string.
for df in [corn_yield_df, df_temp_growing_season, df_precip_growing_season]:
    df['state'] = df['state'].astype(str)
    df['county'] = df['county'].astype(str)
    # Assuming 'year' is consistent (e.g., int) between datasets; if not, convert as needed:
    # df['year'] = df['year'].astype(int)

# Merge corn_yield_df with the temperature dataframe using an outer join.
merged_df = pd.merge(
    corn_yield_df,
    df_temp_growing_season,
    on=['state', 'county', 'year'],
    how='outer'
)

# Merge the resulting dataframe with the precipitation dataframe using an outer join.
merged_df = pd.merge(
    merged_df,
    df_precip_growing_season,
    on=['state', 'county', 'year'],
    how='outer'
)

# Display the first few rows of the merged dataframe
print(merged_df.head())

   year state county division_x                                   data_item  \
0  1926    17    001         03  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   
1  1927    17    001         03  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   
2  1928    17    001         03  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   
3  1929    17    001         03  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   
4  1930    17    001         03  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   

   value  cv division_y    temp_gs division  pcpn_gs  
0   38.0 NaN         03  25.472222       03  831.342  
1   30.0 NaN         03  25.027778       03  605.028  
2   41.0 NaN         03  25.148148       03  540.258  
3   31.0 NaN         03  25.314815       03  725.678  
4   29.0 NaN         03  28.416667       03  352.298  


##### Fix the triple division presence

In [15]:
# Check only rows where division_x is not NaN
mask = merged_df["division_x"].notna()

# Evaluate whether division_x equals division_y in those rows
if (merged_df.loc[mask, "division_x"] == merged_df.loc[mask, "division_y"]).all():
    print("All non-NaN division_x values match division_y.")
else:
    print("Mismatch found between division_x and division_y in some rows.")
    # Optionally, print rows with mismatches for inspection:
    mismatches = merged_df.loc[mask][merged_df.loc[mask, "division_x"] != merged_df.loc[mask, "division_y"]]
    print(mismatches[["division_x", "division_y"]])

Mismatch found between division_x and division_y in some rows.
     division_x division_y
9702         08         09
9703         08         09
9704         08         09
9705         08         09
9706         08         09
...         ...        ...
9794         08         09
9795         08         09
9796         08         09
9798         08         09
9799         08         09

[97 rows x 2 columns]


After inspection of the df it was found that for state 17, county 199, the division is 08 for the yield data and 09 for the NOAA data. A quick inspection of the 'county-to-climdivs' and '240917_corn_yield_data' shows that this is the case from the start and not a coding error.

In [16]:
# Check if all non-NaN values in division_y match division
mask_div = merged_df["division_y"].notna()

if (merged_df.loc[mask_div, "division_y"] == merged_df.loc[mask_div, "division"]).all():
    print("All non-NaN values in division_y match division.")
else:
    print("Mismatch found between division_y and division in some rows.")
    mismatches = merged_df.loc[mask_div][
        merged_df.loc[mask_div, "division_y"] != merged_df.loc[mask_div, "division"]
    ]
    print(mismatches[["division_y", "division"]])

All non-NaN values in division_y match division.


##### Neatly finalize divisions

In [17]:
# Assume merged_df is already created via prior merging steps

# Create a copy of merged_df and work with the new DataFrame final_merged_df
final_merged_df = merged_df.copy()

# 1. Rename 'division_x' to 'division_yield'
final_merged_df.rename(columns={'division_x': 'division_yield'}, inplace=True)

# 2. Create new column 'division_noaa'
# Since you've verified that division_y and division are essentially the same,
# we fill from division_y and fallback to division if necessary.
final_merged_df['division_noaa'] = final_merged_df['division_y'].fillna(final_merged_df['division'])

# 3. Drop the now redundant columns 'division_y' and 'division'
final_merged_df.drop(columns=['division_y', 'division'], inplace=True)

# 4. Reorder columns so that 'division_noaa' is placed immediately after 'division_yield'
cols = list(final_merged_df.columns)
# Find index of 'division_yield'
idx = cols.index('division_yield')
# Build the new column order:
new_cols = cols[:idx+1] + ['division_noaa'] + cols[idx+1:]
# In case 'division_noaa' appears twice, remove duplicates while preserving order
new_cols = list(dict.fromkeys(new_cols))
final_merged_df = final_merged_df[new_cols]

# Display the first few rows to verify the ordering
print(final_merged_df.head())

   year state county division_yield division_noaa  \
0  1926    17    001             03            03   
1  1927    17    001             03            03   
2  1928    17    001             03            03   
3  1929    17    001             03            03   
4  1930    17    001             03            03   

                                    data_item  value  cv    temp_gs  pcpn_gs  
0  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   38.0 NaN  25.472222  831.342  
1  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   30.0 NaN  25.027778  605.028  
2  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   41.0 NaN  25.148148  540.258  
3  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   31.0 NaN  25.314815  725.678  
4  CORN, GRAIN - YIELD, MEASURED IN BU / ACRE   29.0 NaN  28.416667  352.298  


In [18]:
## --- Check if the variable values of the df with new division set-up are the same as before ---

# Define the columns to check
cols_to_check = ['value', 'temp_gs', 'pcpn_gs']

# Compare the corresponding columns between merged_df and final_merged_df
are_equal = merged_df[cols_to_check].equals(final_merged_df[cols_to_check])
print("The 'value', 'temp_gs', and 'pcpn_gs' columns are the same in both DataFrames:", are_equal)

The 'value', 'temp_gs', and 'pcpn_gs' columns are the same in both DataFrames: True


### Save the created df containing both crop yield data and weather regressors.

In [20]:
# Assuming final_merged_df is your final DataFrame that you want to save.
# For example:
# final_merged_df = merged_df  (after doing all your renaming/reordering)

# Convert key columns to string to preserve leading zeros!!!!
for col in ['county', 'division_yield', 'division_noaa']:
    final_merged_df[col] = final_merged_df[col].astype(str)

save_file = "df_yield_climdiv.csv"

final_merged_df.to_csv(save_file, index=False, quoting=csv.QUOTE_ALL)

print("Data saved to df_yield_weather.csv")

Data saved to df_yield_weather.csv
